In [33]:
import pandas as pd
from pathlib import Path
import folium

In [34]:
HEATMAP_CSV = Path('heatmap.csv')

In [35]:
hm = pd.read_csv(HEATMAP_CSV)
hm

,latitude,longitude,intensity
0,33.697423,-117.813692,0.154382
1,33.697423,-117.813649,0.203886
2,33.697423,-117.813606,0.159789
3,33.697387,-117.813735,0.244870
4,33.697387,-117.813692,0.431655
...,...,...,...
11896,33.665997,-117.826695,0.110651
11897,33.665997,-117.826653,0.237577
11898,33.665997,-117.826610,0.310309
11899,33.665997,-117.826567,0.308576


In [36]:
hm_f = hm[hm.intensity > 0.9]

In [37]:
irvine_map = folium.Map(
    zoom_start=12,
    location=[33.669445, -117.823059]
)

for i, r in hm_f.iterrows():
    #setting for the popup
    #Plotting the Marker for each stationsト
    folium.map.Marker(
        location=[r['latitude'], r['longitude']], 
        icon=folium.Icon(color="green",icon="person-running", prefix='fa')
    ).add_to(irvine_map)
    
    folium.vector_layers.CircleMarker(
        location=[r['latitude'], r['longitude']], 
        radius=0.1,
        color='#3186cc',
        fill_color='#3186cc'
    ).add_to(irvine_map)

irvine_map